<a href="https://colab.research.google.com/github/JoungMinJu/lab_study/blob/main/%EC%9B%8C%EB%93%9C%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

임베딩 층 사용하기


In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
sentences=['nice great best amazing','stop lies','pitiful nerd','excellent work','supreme quality','bad','highly respectable']
y_train=[1,0,0,1,1,0,1]

#긍정문장은 1 부정문장은0

In [3]:
t=Tokenizer()
t.fit_on_texts(sentences)#토큰화
vocab_size=len(t.word_index)+1
print(vocab_size)

16


In [4]:
#정수 이코딩
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)


[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [5]:
max_len=max(len(l) for l in X_encoded)
print(max_len)

4


In [6]:
X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train=np.array(y_train)
print(X_train)
#패딩

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [7]:
#위의 과정이 전처리
#아래부터 모델 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model=Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
#출력층에 한개의 뉴런에 활성화 함수로 시그모이드 사용하여 이진분류 수행


In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)


#여기서 말하고자하는게 현재 각 단어들의 임베딩 벡터들의 값은 학습 과정에서 다른 가중치들과 함께 학습된 것이라는 것

Epoch 1/100
1/1 - 1s - loss: 0.6931 - acc: 0.4286
Epoch 2/100
1/1 - 0s - loss: 0.6919 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6906 - acc: 0.7143
Epoch 4/100
1/1 - 0s - loss: 0.6894 - acc: 0.8571
Epoch 5/100
1/1 - 0s - loss: 0.6882 - acc: 0.8571
Epoch 6/100
1/1 - 0s - loss: 0.6871 - acc: 0.8571
Epoch 7/100
1/1 - 0s - loss: 0.6859 - acc: 0.8571
Epoch 8/100
1/1 - 0s - loss: 0.6847 - acc: 0.8571
Epoch 9/100
1/1 - 0s - loss: 0.6835 - acc: 0.8571
Epoch 10/100
1/1 - 0s - loss: 0.6823 - acc: 0.8571
Epoch 11/100
1/1 - 0s - loss: 0.6811 - acc: 0.8571
Epoch 12/100
1/1 - 0s - loss: 0.6799 - acc: 0.8571
Epoch 13/100
1/1 - 0s - loss: 0.6787 - acc: 0.8571
Epoch 14/100
1/1 - 0s - loss: 0.6775 - acc: 0.8571
Epoch 15/100
1/1 - 0s - loss: 0.6763 - acc: 1.0000
Epoch 16/100
1/1 - 0s - loss: 0.6751 - acc: 1.0000
Epoch 17/100
1/1 - 0s - loss: 0.6739 - acc: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.6727 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.6715 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.6703 - a

사전훈련된 Word2Vec 사용하기

In [9]:
import numpy as np
import gensim

In [10]:
# 현재 위치에 구글의 사전 훈련된 Word2Vec을 다운로드
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-08-13 01:44:07--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.129.232
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.129.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  48.0MB/s    in 28s     

2021-08-13 01:44:35 (55.3 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [11]:
# 구글의 사전 훈련된 Word2vec 모델을 로드합니다.
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [12]:
print(word2vec_model.vectors.shape) # 모델의 크기 확인

#300의 차원을 가진  Word2Vec 벡터가 3,000,000개 있다.


(3000000, 300)


In [13]:
embedding_matrix = np.zeros((vocab_size, 300))
# 단어 집합 크기의 행과 300개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

#모든 값이 0으로 채워진 임베딩 행렬. 이번 문제의 단어는 총 16개이므로 16x300의 크기를 가진 행렬을 만든다

(16, 300)

In [16]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [17]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어와 정수 인덱스를 1개씩 꺼내온다.
    temp = get_vector(word) # 단어(key) 해당되는 임베딩 벡터의 300개의 값(value)를 임시 변수에 저장
    if temp is not None: # 만약 None이 아니라면 임베딩 벡터의 값을 리턴받은 것이므로
        embedding_matrix[i] = temp # 해당 단어 위치의 행에 벡터의 값을 저장한다.

In [18]:
#word2vec_model에 저장되어있던 단어 'nice'의 임베딩 벡터 값 확인
print(word2vec_model['nice'])

[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.12792969  0.09619141
 -0.12158203 -0.08691406 -0.12890625  0.27734375  0.265625    0.1796875
  0.12695312  0.06298828 -0.34375    -0.05908203  0.0456543   0.171875
  0.08935547  0.14648438 -0.04638672 -0.00842285 -0.0279

In [19]:
#nice가 가지고 있는 인덱스
print('nice의 정수 인덱스 : ',t.word_index['nice'])
print(embedding_matrix[t.word_index['nice']])

nice의 정수 인덱스 :  1
[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.12792969  0.09619141
 -0.12158203 -0.08691406 -0.12890625  0.27734375  0.265625    0.1796875
  0.12695312  0.06298828 -0.34375    -0.05908203  0.0456543   0.171875
  0.08935547  0.14648438 -0.04638672 -

In [21]:
#Embedding에 사전 훈련된 embedding matrix를 입력으로 넣어주고 모델을 학습시킴
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input

model=Sequential()
model.add(Input(shape=(max_len,),dtype='int32'))
e=Embedding(vocab_size, 300, weights=[embedding_matrix],input_length=max_len,trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.7166 - acc: 0.4286
Epoch 2/100
1/1 - 0s - loss: 0.6977 - acc: 0.4286
Epoch 3/100
1/1 - 0s - loss: 0.6794 - acc: 0.4286
Epoch 4/100
1/1 - 0s - loss: 0.6616 - acc: 0.7143
Epoch 5/100
1/1 - 0s - loss: 0.6444 - acc: 0.7143
Epoch 6/100
1/1 - 0s - loss: 0.6277 - acc: 0.7143
Epoch 7/100
1/1 - 0s - loss: 0.6116 - acc: 0.7143
Epoch 8/100
1/1 - 0s - loss: 0.5960 - acc: 0.8571
Epoch 9/100
1/1 - 0s - loss: 0.5809 - acc: 1.0000
Epoch 10/100
1/1 - 0s - loss: 0.5664 - acc: 1.0000
Epoch 11/100
1/1 - 0s - loss: 0.5523 - acc: 1.0000
Epoch 12/100
1/1 - 0s - loss: 0.5387 - acc: 1.0000
Epoch 13/100
1/1 - 0s - loss: 0.5255 - acc: 1.0000
Epoch 14/100
1/1 - 0s - loss: 0.5128 - acc: 1.0000
Epoch 15/100
1/1 - 0s - loss: 0.5005 - acc: 1.0000
Epoch 16/100
1/1 - 0s - loss: 0.4886 - acc: 1.0000
Epoch 17/100
1/1 - 0s - loss: 0.4771 - acc: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.4660 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.4552 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.4448 - a